In [36]:
## STANDARDIZZATORE ##

#FASI
#1. LETTURA estensione
#2. TRADUZIONE in JSON


#FORMATI DATASET (fonte: https://docs.italia.it/AgID/documenti-in-consultazione/lg-opendata-docs/it/bozza/allegato-b-standard-di-riferimento-e-formati-aperti.html)

#!! PRINCIPALI !!
##Formati aperti per i dati
#CSV (Comma Separated Values)
#JSON (JavaScript Object Notation)
#XML (eXtensible Markup Language)
#XLSX (Excel)
#!!


##Formati aperti più diffusi per i dati geografici
#Shapefile
#KML
#GeoJSON
#GML (Geography Markup Language)
#GeoPackage

##Formati aperti per i documenti
#ODF (Open Document Format)
#PDF
#Akoma Ntoso

##Formati per dati meteorologici
#BUFR (Binary Universal Form for the Representation of meteorological data)
#NetCDF (Network Common Data Form)
#ASCII (American Standard Code for Information Interchange)
#Avvisi Meteo: 
#CAP (Common Alerting Protocol), RSS (Really Simple Syndication)/Atom
#Radar: 
#HDF5 (Hierarchical Data Format)
#Modello NWP (Numerical weather prediction): 
#GRIB (General Representation of fields In Binary)


In [1]:
import pandas as pd
import json
import os
import xml.etree.ElementTree as ET

def extract(file_path):
    root, extension = os.path.splitext(file)
    columns_list = []
    if (extension == '.csv'):
        columns_list = extract_csv(file_path)      
    elif (extension == '.xml'):
        columns_list = extract_xml(file_path) 
    elif (extension == '.json'):
        columns_list = extract_json(file_path)
    elif (extension == '.xlsx'):
        columns_list = extract_excel(file_path)      
        
    return columns_list
        
def extract_csv(file_path):
    data = pd.read_csv(file_path, encoding='latin1')         
    columns_list = [col for col in data.columns.tolist() if not col.startswith('Unnamed')]
    return columns_list
    
def extract_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Filtra le chiavi con nomi diversi da 'Unnamed'
    columns_list = [key for key in data.keys() if not key.startswith('Unnamed')]
    return columns_list

def extract_excel(file_path):
    data = pd.read_excel(file_path)
    # Filtra le colonne con nomi diversi da 'Unnamed'
    columns_list = [col for col in data.columns.tolist() if not col.startswith('Unnamed')]
    return columns_list

def extract_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    columns = set()

    # esplorazione dell'XML
    def explore_xml(element):
        nonlocal columns
        if not element.tag.startswith('Unnamed'):
            columns.add(element.tag)  # Aggiungi il nome del tag come colonna
        for child in element:
            explore_xml(child)  # Esplora ricorsivamente gli elementi figlio

    # Esplora ricorsivamente l'XML per identificare le colonne
    explore_xml(root)

    columns_list = list(columns)
    
    return columns_list
 


In [8]:
#Directory
root_directory = 'test dataset clustering'
extensions = ['.csv', '.xml', '.json', '.xlsx']
directories = [dirc for dirc in os.listdir(root_directory)]
#files = [file for file in os.listdir(directory) if file.endswith(tuple(extensions))]

# Dizionario per contenere i dati convertiti in JSON
datasets_json = {}

# Converte ciascun file in un JSON
for directory in directories:
    category = directory
    directory_path = root_directory + "/" + directory
    files = [file for file in os.listdir(directory_path) if file.endswith(tuple(extensions))]
    for file in files:
        file_path = directory_path + "/" + file
        #toglie estensione
        dataset_name = os.path.splitext(file)[0]
        #print(dataset_name)
        #splitta da categoria con simbolo $
        #split_name = dataset_name.split('$', 1)
        #real_name = split_name[0]
        #category = split_name[1]
        
        columns_list = extract(file_path)
        columns_list.insert(0,category)
        #print(columns_list[0:10])
        # Aggiunge al dizionario JSON
        datasets_json[dataset_name] = columns_list
        

# Esporta il dizionario in formato JSON
with open("export.json", "w") as outfile:
    json.dump(datasets_json, outfile, indent=4,)

#print(datasets_json)

C:\Users\acer\AppData\Local\Temp\ipykernel_3812\34338451.py:21: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, encoding='latin1')
